In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Load data

In [27]:
datadir = 'Behavioral-Analyzed-Data/training/'

In [28]:
folders=[datadir+s for s in os.listdir(datadir) if len(s) <= 2]
len(folders)

20

In [40]:
subjects = pd.DataFrame(folders,columns=['folder'])
subjects.index = [int(s.split('/')[-1]) for s in subjects.folder]
subjects.sort_index()
subjects

,folder
1,Behavioral-Analyzed-Data/training/1
2,Behavioral-Analyzed-Data/training/2
3,Behavioral-Analyzed-Data/training/3
4,Behavioral-Analyzed-Data/training/4
5,Behavioral-Analyzed-Data/training/5
6,Behavioral-Analyzed-Data/training/6
7,Behavioral-Analyzed-Data/training/7
8,Behavioral-Analyzed-Data/training/8
9,Behavioral-Analyzed-Data/training/9
10,Behavioral-Analyzed-Data/training/10


# Dimensionality reduction

# Directionality modelling

# Results